In [1]:
#Todos os imports necessários para aplicação.


import os
import librosa
import numpy as np
import sagemaker_pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField,StringType, FloatType

In [11]:
from pyspark import SparkContext, SparkConf
from sagemaker_pyspark import classpath_jars
from pyspark.sql.functions import create_map, struct
from pydub import AudioSegment
from soundscrape.soundscrape import process_soundcloud

In [3]:
LANDED = '/home/rlfo/Documents/pessoal/bigdatamusic/landed'
RAW = '/home/rlfo/Documents/pessoal/bigdatamusic/raw'

In [7]:
classpath = ":".join(sagemaker_pyspark.classpath_jars())

builder = SparkSession.builder.appName("MUSIC SPARK")
builder.config(
    "spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2")
builder.config("spark.speculation", "false")
builder.config("spark.sql.parquet.compression.codec", "gzip")
builder.config("spark.debug.maxToStringFields", "100")
builder.config("spark.driver.extraClassPath", classpath)
builder.config("spark.driver.memory", "1g")
builder.config("spark.driver.cores", "1")
builder.config("spark.executor-memory", "20g")
builder.config("spark.executor.cores", "4")


builder.master("local[*]")

spark = builder.getOrCreate()
spark


In [8]:
all_artist = ['https://soundcloud.com/soundcloud-hustle/sets/payday','https://soundcloud.com/helennpham/sets/lofi-hip-hop','https://soundcloud.com/lukasm1/sets/chill-mix-high-on-chill']

In [9]:
def download_audio(url='', number=4, path=LANDED):
    vargs = {'path':path, 'folders': False, 'group': False, 'track': '', 'num_tracks': number, 'bandcamp': False, 'downloadable': False, 'likes': True, 'open': False, 'artist_url': url, 'keep': True}
    process_soundcloud(vargs)

In [12]:
pipe_music = spark.sparkContext.parallelize(all_artist).foreach(download_audio)